# Text Prompt Segmentation

This example demonstrates how to use the `geoai-py` package for text prompt segmentation using [CLIPSeg](https://huggingface.co/docs/transformers/v4.49.0/en/model_doc/clipseg#overview).

[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/docs/examples/text_prompt_segmentation.ipynb)

## Install package

To use the `geoai-py` package, ensure it is installed in your environment. Uncomment the command below if needed.

In [20]:
%pip install geoai-py

## Import libraries

In [21]:
import geoai

## Download sample data

In [22]:
raster_url = (
    "/content/before.tif"
)
raster_path = geoai.download_file(raster_url)

File already exists: before.tif


## Visualize data

In [23]:
geoai.view_raster(raster_url)

## Initialize model

In [24]:
segmenter = geoai.CLIPSegmentation(tile_size=512, overlap=32)

Model loaded on cuda


## Run inference

In [25]:
output_path = "tree_masks1.tif"
text_prompt = "trees"

In [26]:
segmenter.segment_image(
    raster_path,
    output_path=output_path,
    text_prompt=text_prompt,
    threshold=0.5,
    smoothing_sigma=1.0,
)

Processing tiles: 100%|██████████| 391/391 [00:25<00:00, 15.06it/s]


Segmentation saved to tree_masks1.tif


'tree_masks1.tif'

In [27]:
import rasterio
from rasterio import features
import geopandas as gpd
from shapely.geometry import shape

# INPUTS
mask_raster = "tree_masks1.tif"
output_geojson = "tree_masks1_4326.geojson"

# Step 1: Read the segmentation mask and metadata
with rasterio.open(mask_raster) as src:
    mask = src.read(1)  # Assumes binary mask (0/1)
    transform = src.transform
    crs = src.crs

# Step 2: Extract polygons from raster mask
polygons = []
for geom, val in features.shapes(mask, mask=mask == 1, transform=transform):
    if val == 1:
        polygons.append(shape(geom))

# Step 3: Create GeoDataFrame in original CRS
gdf = gpd.GeoDataFrame(geometry=polygons, crs=crs)

# Step 4: Reproject to EPSG:4326
gdf_4326 = gdf.to_crs(epsg=4326)

# Step 5: Save GeoJSON
gdf_4326.to_file(output_geojson, driver="GeoJSON")

print("GeoJSON saved:", output_geojson)


GeoJSON saved: tree_masks1_4326.geojson


## Visualize results

In [28]:
geoai.view_raster(
    output_path,
    nodata=0,
    opacity=0.8,
    colormap="greens",
    layer_name="Trees",
    basemap=raster_url,
)

In [30]:
geoai.create_split_map(
    left_layer=output_path,
    right_layer=raster_url,
    left_label="Trees",
    right_label="Satellite Image",
    left_args={"nodata": 0, "opacity": 0.8, "colormap": "greens"},
    basemap=raster_url,
)

Map(center=[23.6358235, 86.2879155], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

![image](https://github.com/user-attachments/assets/8fa8803e-072b-40cf-bfa8-a57ead1805e8)